## Segmenting and Clustering Neighborhoods in Toronto

First I import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import urllib.request as request
from bs4 import BeautifulSoup

Next I scrap the page

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = request.urlopen(url)
soup = BeautifulSoup(page, "lxml")

#print(soup.prettify())

I find the table, extract the data, and convert it into a data frame

In [3]:
right_table=soup.find('table', class_='wikitable sortable')
#right_table

In [4]:
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [5]:
df=pd.DataFrame(A,columns=['Postcode'])
df['Borough']=B
df['Neighborhood']=C
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


I remove rows where borough is 'Not assigned' and reset the index

In [6]:
df = df[df.Borough != 'Not assigned']
df.index = [x for x in range(0, len(df.values))]
df['Neighborhood'] = df['Neighborhood'].str.replace('\n','')

In [7]:
df.describe()

,Postcode,Borough,Neighborhood
count,210,210,210
unique,103,10,208
top,M8Y,Etobicoke,Runnymede
freq,8,45,2


This code replaces 'Not Assigned' neighborhoods with the borough name

In [8]:
x=0
while x < 210:
    if df.iat[x, 2] == 'Not Assigned':
        df.iat[x, 2] = df.iat[x, 1]
    x = x+1

In [9]:
aggregation_functions = {'Borough':'first','Neighborhood':', '.join}
df_new = df.groupby(df['Postcode']).aggregate(aggregation_functions)
df_new.head()

,Borough,Neighborhood
Postcode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [10]:
df_new.shape

(103, 2)